# Les Versions de l'article 9

In [6]:
#import des modules

#lecture des json
import json 

#affichage des valeurs identées
import pprint
pp = pprint.PrettyPrinter(indent=4, )

#affichage des graphs
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [7]:

#chargement de l'article 9
with open("article9.json", "r") as f:
    article9 = json.load(f)

* On va stocker le détail des participations de l'article 9 dans version_originale de la manière suivante:

In [8]:
version_originale = {
                "date": article9["created_at"],
                "link": article9["article_link"],
                "slug": article9["article_link"].split("/")[-1],
                "title": article9["article_link"].split("/")[-1].replace("-", " "),
                "text": article9['body'],
                "author": article9["author"],
                "votes":[], 
                "arguments":article9["arguments"], 
                "votes_arguments": [],
                "sources":article9["sources"],
                "votes_sources": [],
               }

* On va stocker les versions complémentaires dans versions

In [9]:
versions = article9["versions"]


et reformater les versions de la même manière que dans la version originale


In [10]:
versions_c = []
for v in versions:
    vn = {"date":v["created_at"],
"link":v["link"],
"slug":v["slug"],
"title":v["title"],
"text": v['comment'],
"author": v["author"],
          "votes":[], 
"arguments":[], 
"votes_arguments": [],
"sources":[],
"votes_sources": []}
    versions_c.append(vn)

* On ajoute ensuite la version originale

In [11]:
versions_c.append(version_originale)

* Maintenant on va le transformer en dictionnaire avec pour clé le slug

In [12]:
versions_d = {}
for v in versions_c:
    versions_d[v["slug"]] = v
    


In [13]:
print len(versions_d)

109


# Les electeurs de l'article 9

On charge le fichiers participants qui contient toutes les actions de chaque participant

In [14]:

#Ici on ne s'intéresse qu'aux votes
#qui sont détaillés par participants
with open("participants.json", "r") as f:
    data = json.load(f)
    participants = data["participants"]
#Verifions le nombre de participants

print len(participants), "participants sur l'ensemble des articles"
#les noms des participants
electeurs = list(set([n.keys()[0] for n in participants]))


21329 participants sur l'ensemble des articles


On charge ensuite seulement les votes des électeurs de l'article 9 dans votes

In [15]:
art = "article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics"
votes = []
electeurs_d = {}
for user in participants:
    #accesible via son nom
    name =  user.keys()[0]
    electeurs_d[name] = []
    #récupérer les votes de l'utilisateur dans "votes"
    #filtrer ceux qui ont des votes qui correspondent à l'article
    for vote in user[name]["votes"]:
        if art in vote["link"]:
            vote["electeur"] = name
            electeurs_d[name].append(vote)
            if vote not in votes:
                votes.append(vote)
            

In [16]:
print len(votes), "votes sur des articles, arguments ou sources"
print len(electeurs_d), "electeurs"

15547 votes sur des articles, arguments ou sources
21329 electeurs


In [17]:
for vote in votes:
    url = vote["link"]
    slug = url.split("/")[-1]
    #vote sur un argument        
    if "#arg" in slug:
        vote["slug"], vote["id"] = slug.split("#")
        versions_d[vote["slug"]]["votes_arguments"].append(vote)
        
        #vote sur une source
    elif "#source" in slug:
        vote["slug"], vote["id"] = slug.split("#")
        versions_d[vote["slug"]]["votes_sources"].append(vote)
        
    #vote sur un article
    else:
        vote["slug"] = slug
        versions_d[vote["slug"]]["votes"].append(vote)
        

In [18]:
print len(versions_d[art]["votes"])

3334


In [19]:
print len(versions_d[art]["sources"])

22


In [20]:
print len(versions_d[art]["arguments"])

132


In [21]:
print len(versions_d[art]["votes_sources"])

130


In [22]:
print len(versions_d[art]["votes_arguments"])

1155


In [23]:
auteur_electeur = []

for version, data in versions_d.items():
    for vote in data["votes"]:
        auteur_electeur.append([version, data["author"], vote["electeur"], vote["opinion"]])


In [24]:
print len(auteur_electeur)

12851


In [25]:
with open("votes_auteur_electeur.csv", "w") as f:
    f.write("version\tauteur\telecteur\tvote\n")
    for line in auteur_electeur:
        f.write(("\t").join(line)+"\n")
        

Reseau d'affinité electeur auteur
Deux acteurs sont reliés dès lors que l'un d'entre eux a voté **pour** une version proposé par l'autre.

Matrice de coocurrence: on multiplie la matrice par sa transposition
de sorte que f(x) = x(f)
soit a->b et b->a ou encore 
paul a voté pour pierre & pierre a voté pour paul
on doit garantir que dans notre résultat les deux formes ne cooexistent pas
soit la diagonale doit etre enlevé ou encore garantir qu'une seule des options de la combinaison existe



# Réseau d'affinité entre electeurs

Deux electeurs sont reliés quand ils ont votés de manière similaires selon 
un score de similiarité de comportement de votes

Avant de procéder à cette analyse regardons donc la répartition de participations des électeurs

Un nombre de votes par electeur situé entre 1 et 102

On a 5635 electeurs pour un total de 12851 votes.

On va d'abord se concentrer sur les electeurs qui ont votés plusieurs fois.
Et ensuite chercher un seuil minimal de participation pour détecter la similarité entre 
deux electeurs.

En effet si un electeur a voté une seule fois et qu'un autre electeur 15 fois
leur comportement est similaire alors que ce n'est pas révélateur

In [26]:
from collections import Counter
import operator
votes_electeurs = Counter([n[2] for n in auteur_electeur])
nb_electeurs = len(votes_electeurs)
nb_votes = sum(votes_electeurs.values())
print nb_votes,"votes", nb_electeurs, "electeurs"
#top_votants = votes_electeurs.most_common(563)

#print top_votants

#plt.bar(range(len(top_votants)), top_votants.values(), align='center')
#plt.xticks(range(len(top_votants)), range(len(top_votants)))
#plt.show()

one_vote = {k:v for k,v in votes_electeurs.items() if v ==1}
nb_one_vote = len(one_vote)
print nb_one_vote, "electeurs uniques qui ont voté une seule fois"
multiple_vote = {k:v for k,v in votes_electeurs.items() if v > 1}
print len(multiple_vote), "electeurs uniques qui ont voté plusieurs fois"
print "pour un total de", sum(multiple_vote.values()), "votes"
#seuil de participations = 5
votes_seuil = {k:v for k,v in votes_electeurs.items() if v >= 5}
print len(votes_seuil), sum(votes_seuil.values())


12851 votes 5635 electeurs
3043 electeurs uniques qui ont voté une seule fois
2592 electeurs uniques qui ont voté plusieurs fois
pour un total de 9808 votes
482 4574


In [27]:
from itertools import combinations
from math import factorial as fact
combo = list(set(combinations(votes_seuil.keys(), 2)))

print len([n for n in combo])

def combo_score(n,k):
    #en combinatoire calculer le nombre de combinaison de 2 elements sur un ensemble de 482 elements
    #https://fr.wikipedia.org/wiki/Combinaison_(math%C3%A9matiques)
    #combin = lambda n,k: fact(n)//(fact(k)*fact(n-k))
    return fact(n)/(fact(k)*fact(n-k))
#On vérifie
print combo_score(482, 2)


115921
115921


In [ ]:

similarity_score = {}

for c in combinations(votes_seuil.keys(), 2):
    #print c
    userA, userB = c
    #print userA, userB
    similarity_score[c] = 0
    userweight = 0
    for v, k in versions_d.items():
        electeur_d = {e["electeur"]:e["opinion"] for e in k["votes"]}
        #print electeur_d.keys()
        if userA in electeur_d.keys() and userB in electeur_d.keys():
            if electeur_d[userA] == electeur_d[userB]:
                similarity_score[c] += 1
    #print c, similarity_score[c]      


* Construire le graphe
Chaque noeud correspond à une personne 
Deux electeurs sont reliés si leur score de similarite est > 0 
et la taille du lien dépend du score 


In [26]:
import networkx as nx
g = nx.Graph()
pos = nx.balanced_tree(3,5)
g.add_nodes_from(votes_seuil.keys(), font_size = 16)
#nx.draw_networkx_labels(g,pos,votes_seuil,font_size=16)

#plt.show()
edges = [(user_co[0], user_co[1]) for user_co, score in similarity_score.items() if score > 0]
#, {"color":"green", "weight":score})
g.add_edges_from(edges)
nx.write_gexf(g, "electeurs.gexf")
#nx.draw(g, with_labels=True)
#plt.savefig('graph.png')
#plt.show()
  

Evidemment on ne voit rien

# Réseau d'affinité entre l'auteur et ses electeurs:

Deux acteurs sont reliés lorsque que l'un à voté *pour* l'autre


In [ ]:
auteur_vs_electeur = { "auteur":[], "electeur":[], "vote":[], "version":[]}
for line in auteur_electeur:
    auteur_vs_electeur["version"].append(line[0])
    auteur_vs_electeur["auteur"].append(line[1])
    auteur_vs_electeur["electeur"].append(line[2])
    auteur_vs_electeur["vote"].append(line[3])
    

In [ ]:
from collections import defaultdict
#dictionnaire de auteur -> electeur
elu_par = defaultdict.fromkeys(set(auteur_vs_electeur["auteur"]), [])
#dictionnaire de electeur -> version
a_vote_pour = defaultdict.fromkeys(set(auteur_vs_electeur["electeur"]), [])
votes_pour = []
with open("a_vote_pour.csv","w") as f:
    f.write("auteur\telecteur\tversion\n")
    for line in auteur_electeur:
        auteur = line[1]
        electeur = line[2]
        version = line[0]
        if line[3] == "1":
            
            votes_pour.append(line)
            f.write("\t".join(line[0:3])+"\n")
            elu_par[auteur].append(electeur)              
            a_vote_pour[electeur].append(version)
        


In [ ]:
import networkx as nx

In [ ]:
def find_isolated_nodes(graph):
    """ returns a list of isolated nodes. """
    isolated = []
    for node in graph:
        if not graph[node]:
            isolated += node
    return isolated
isolated = find_isolated_nodes(elu_par)
print isolated

les noeuds seront les auteurs
et les edges les votes pour

In [ ]:
G = nx.Graph()
#G.add_nodes_from(elu_par.keys(), type="auteur")


for k,v in elu_par.items():
    nx.set_node_attributes(G,k, k)
    for v2 in v:
        G.add_edge(k,v2)

nx.draw(G)
plt.show()


In [ ]:
G = nx.Graph()